In [564]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import linear_model
import random
import csv
from sklearn import svm

In [515]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [516]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [517]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### Force `NaN` to 0, add `isNaN` column

In [518]:
for item in list(df):
    if df.loc[df[item].isnull()].shape[0]>0:
        addname = item+'.isNaN'
        df[addname] = np.where(df[item].isnull(),1,0)
        df[item]=np.where(df[item].isnull(),0,df[item])
        print(item, ': ',df.loc[df[item].isnull()].shape[0])

Age :  0
Cabin :  0
Embarked :  0


In [519]:
for item in list(df_test):
    if df_test.loc[df_test[item].isnull()].shape[0]>0:
        addname = item+'.isNaN'
        df_test[addname] = np.where(df_test[item].isnull(),1,0)
        df_test[item]=np.where(df_test[item].isnull(),0,df_test[item])
        print(item, ': ',df_test.loc[df_test[item].isnull()].shape[0])

Age :  0
Fare :  0
Cabin :  0


In [520]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age.isNaN,Cabin.isNaN,Embarked.isNaN
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S,0,1,0


In [521]:
df.shape

(891, 15)

In [522]:
df_test.shape

(418, 14)

In [523]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age.isNaN,Fare.isNaN,Cabin.isNaN
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,0,Q,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,0,S,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,0,Q,0,0,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,0,S,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,0,S,0,0,1


In [524]:
df.groupby('Pclass')['PassengerId'].count()

Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64

In [525]:
df.groupby('SibSp')['PassengerId'].count()

SibSp
0    608
1    209
2     28
3     16
4     18
5      5
8      7
Name: PassengerId, dtype: int64

In [526]:
df.groupby('Parch')['PassengerId'].count()

Parch
0    678
1    118
2     80
3      5
4      4
5      5
6      1
Name: PassengerId, dtype: int64

In [527]:
df.groupby('Sex')['PassengerId'].count()

Sex
female    314
male      577
Name: PassengerId, dtype: int64

In [528]:
AgeQuan = list(df['Age'].quantile([0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]))
print(AgeQuan)

[0.0, 6.0, 19.0, 24.0, 29.0, 35.0, 45.0, 80.0]


In [529]:
def getAgeCategory(ageList):
    print(len(ageList))
    ageCate = []
    for age in (ageList):
        if age<=AgeQuan[0]:
            ageCate.append('A')
        elif age<=AgeQuan[1]:
            ageCate.append('B')
        elif age<=AgeQuan[2]:
            ageCate.append('C')
        elif age<=AgeQuan[3]:
            ageCate.append('D')
        elif age<=AgeQuan[4]:
            ageCate.append('E')
        elif age<=AgeQuan[5]:
            ageCate.append('F')
        elif age<=AgeQuan[6]:
            ageCate.append('G')
        elif age<=AgeQuan[7]:
            ageCate.append('H')
        else:
            ageCate.append('I')
    return ageCate

In [530]:
df['AgeCate']  = getAgeCategory(df['Age'])

891


In [531]:
df_test['AgeCate']  = getAgeCategory(df_test['Age'])

418


In [532]:
df.groupby('AgeCate')['PassengerId'].count()

AgeCate
A    177
B     47
C    117
D    113
E    107
F    113
G    114
H    103
Name: PassengerId, dtype: int64

In [533]:
df_test.groupby('AgeCate')['PassengerId'].count()

AgeCate
A    86
B    15
C    46
D    70
E    54
F    42
G    53
H    52
Name: PassengerId, dtype: int64

In [534]:
FareCate = list(df['Fare'].quantile([0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]))
print(FareCate)

[7.75, 7.9104000000000001, 9.8406500000000001, 14.4542, 24.479174999999998, 31.0, 69.487499999999997, 512.32920000000001]


In [535]:
len(FareCate)

8

In [536]:
def getFareCate(fareList):
    fareCate = []
    print(len(list(fareList)))
    for fare in list(fareList):
        if fare<FareCate[0]:
            fareCate.append('A')
        elif fare<FareCate[1]:
            fareCate.append('B')
        elif fare<FareCate[2]:
            fareCate.append('C')
        elif fare<FareCate[3]:
            fareCate.append('D')
        elif fare<FareCate[4]:
            fareCate.append('E')
        elif fare<FareCate[5]:
            fareCate.append('F')
        elif fare<FareCate[6]:
            fareCate.append('G')
        elif fare<FareCate[7]:
            fareCate.append('H')
        else:
            fareCate.append('I')
    return fareCate

In [537]:
df['FareCate']  = getFareCate(df['Fare'])
df_test['FareCate']  = getFareCate(df_test['Fare'])

891
418


In [538]:
df.groupby('FareCate')['PassengerId'].count()

FareCate
A    106
B    117
C    111
D    106
E    117
F    109
G    113
H    109
I      3
Name: PassengerId, dtype: int64

In [539]:
df_test.groupby('FareCate')['PassengerId'].count()

FareCate
A    50
B    65
C    41
D    52
E    57
F    45
G    53
H    54
I     1
Name: PassengerId, dtype: int64

In [540]:
df['CabinCate'] = df['Cabin'].apply(lambda x: str(x)[0])
df_test['CabinCate'] = df_test['Cabin'].apply(lambda x: str(x)[0])

In [541]:
df.groupby('CabinCate')['PassengerId'].count()

CabinCate
0    687
A     15
B     47
C     59
D     33
E     32
F     13
G      4
T      1
Name: PassengerId, dtype: int64

In [542]:
df_test.groupby('CabinCate')['PassengerId'].count()

CabinCate
0    327
A      7
B     18
C     35
D     13
E      9
F      8
G      1
Name: PassengerId, dtype: int64

In [543]:
label = df['Survived']

In [544]:
passId = df_test['PassengerId']

In [545]:
delList = ['Name', 'Ticket','Cabin', 'Survived', 'PassengerId']

In [546]:
for col in df:
    if col in delList:
        del df[col]

In [547]:
for col in df_test:
    if col in delList:
        del df_test[col]

In [548]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age.isNaN,Cabin.isNaN,Embarked.isNaN,AgeCate,FareCate,CabinCate
0,3,male,22.0,1,0,7.2500,S,0,1,0,D,A,0
1,1,female,38.0,1,0,71.2833,C,0,0,0,G,H,C
2,3,female,26.0,0,0,7.9250,S,0,1,0,E,C,0
3,1,female,35.0,1,0,53.1000,S,0,0,0,F,G,C
4,3,male,35.0,0,0,8.0500,S,0,1,0,F,C,0


In [549]:
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age.isNaN,Fare.isNaN,Cabin.isNaN,AgeCate,FareCate,CabinCate
0,3,male,34.5,0,0,7.8292,Q,0,0,1,F,B,0
1,3,female,47.0,1,0,7.0000,S,0,0,1,H,A,0
2,2,male,62.0,0,0,9.6875,Q,0,0,1,H,C,0
3,3,male,27.0,0,0,8.6625,S,0,0,1,E,C,0
4,3,female,22.0,1,1,12.2875,S,0,0,1,D,D,0


In [550]:
def logit_cv(data,label):
    logreg = linear_model.LogisticRegression(fit_intercept=True, C=1e5)
    kf = KFold(n_splits=5, shuffle = True)
    folds = kf.split(data)
    score = []
    for train,test in folds:
        data_train, data_test = data[train],data[test]
        label_train, label_test = label[train],label[test]
        logreg.fit(data_train, label_train)
        Z = logreg.score(data_test, label_test)
        score.append(Z)
    return [np.mean(score),logreg]

In [551]:
def forward_fea_select_logit(data, label, n):
    colist = list(data)
    selist = []
    left = len(colist)-n
    cm = []
    while (len(colist)>left):
        maxfeaval = 0
        temp = 0
        maxfea = ''
        for col in colist:
            selist.append(col)
            fea = preprocessing.scale(data[selist])
            selist.remove(col)
            [temp,logreg] = logit_cv(fea,label)
            if temp>maxfeaval:
                maxfeaval = temp
                maxfea = col
        colist.remove(maxfea)
        selist.append(maxfea)
        print('Feature List:',selist)
        print('Accuracy: %.2f'%maxfeaval)
    return [selist,logreg]

In [552]:
df_dummy = pd.get_dummies(df)

In [553]:
N = 8
[selist,logreg] = forward_fea_select_logit(df_dummy, label, N)

Feature List: ['Sex_female']
Accuracy: 0.79
Feature List: ['Sex_female', 'AgeCate_B']
Accuracy: 0.79
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp']
Accuracy: 0.80
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp', 'Pclass']
Accuracy: 0.82
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp', 'Pclass', 'Parch']
Accuracy: 0.82
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp', 'Pclass', 'Parch', 'FareCate_B']
Accuracy: 0.82
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp', 'Pclass', 'Parch', 'FareCate_B', 'CabinCate_F']
Accuracy: 0.82
Feature List: ['Sex_female', 'AgeCate_B', 'SibSp', 'Pclass', 'Parch', 'FareCate_B', 'CabinCate_F', 'AgeCate_F']
Accuracy: 0.82


#### SVM RBF Kernel

In [641]:
def svm_cv(data,label):
    C = 1.0 
    #clf = svm.SVC(kernel='rbf', gamma=0.1, C=C)
    #clf = svm.SVC(kernel='poly', degree=3, C=C)
    clf = svm.SVC(kernel='linear', C=C)
    kf = KFold(n_splits=5, shuffle = True)
    folds = kf.split(data)
    score = []
    for train,test in folds:
        data_train, data_test = data[train],data[test]
        label_train, label_test = label[train],label[test]
        clf.fit(data_train, label_train)
        Z = clf.score(data_test, label_test)
        score.append(Z)
    return [np.mean(score),clf]

In [642]:
#C = 1.0  # SVM regularization parameter
# models = (svm.SVC(kernel='linear', C=C),
#           svm.LinearSVC(C=C),
#           svm.SVC(kernel='rbf', gamma=0.7, C=C),
#           svm.SVC(kernel='poly', degree=3, C=C))
# models = (clf.fit(X, y) for clf in models)
#clf = svm.SVC(kernel='rbf', gamma=0.7, C=C)

In [643]:
data = preprocessing.scale(df_dummy)

In [644]:
[res, clf] = svm_cv(data, label)

In [645]:
print(res)

0.812516477308


#### Predict test data

In [554]:
df_test_dummy = pd.get_dummies(df_test)

In [555]:
df_test_dummy.shape

(418, 38)

In [556]:
data_test = preprocessing.scale(df_test_dummy[selist])

In [558]:
res = list(logreg.predict(data_test))

In [559]:
print(res)

[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

In [561]:
passId = list(passId)
print(passId)

[892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 107

In [562]:
passId.insert(0, "PassengerId")
res.insert(0, "Survived")
result = [passId, res]
print(result[0][:10])
print(result[1][:10])

['PassengerId', 892, 893, 894, 895, 896, 897, 898, 899, 900]
['Survived', 0, 1, 0, 0, 1, 0, 1, 0, 1]


In [563]:
with open('output.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(np.transpose(result))